In [1]:
import numpy as np
import scipy as sc
import pylab as py
import gzip
import math
from scipy.optimize import fmin_bfgs, fmin_l_bfgs_b, fmin_cg
from collections import defaultdict
import heapq
import copy
import bigfloat
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import operator
import csv
import hashlib
from math import exp
from math import log
import warnings
warnings.filterwarnings("error")
from datetime import datetime
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [2]:
with open('categories.txt', 'rb') as content_file:
    categories = content_file.read().split('\n')
#to make things faster i put the index where every category starts in a dictionary where the key are the product ids and 
#the values are the starting index of the category
catindex = defaultdict(int)
f = open('categories.txt','rb')
for i,l in enumerate(f):
    if len(l)-len(l.strip()) <=1:
        catindex[l.strip()]=i

In [3]:
def getYear(unix):
    time = datetime.utcfromtimestamp(float(unix))
    return time.strftime("%Y")

In [4]:
#takes product id and returns category
def getCategory(pid, categories, catdict, depth):
    index = catdict[pid]
    value = set()
    j = index + 1
    while len(categories[j])-len(categories[j].strip())>1:
                ws = (categories[j].strip()).split(',')
                break
    return str((ws[:depth])[-1].strip())

In [77]:
data = []
with open('500 core subset filtered.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append((row['uid'], row['pid'], row['rating'], row['time']))

In [78]:
def gethexdig(txt):
    c = hashlib.md5(txt.encode())
    return c.hexdigest()

In [79]:
def get_top_genres(data):
    gs= defaultdict(int)
    for d in data:
        cat = getCategory(d[1], categories, catindex, 3)
        gs[cat]+=1
    return set(sorted(gs.iteritems(), key= operator.itemgetter(1), reverse=True)[:50])

In [80]:
gtop = get_top_genres(data)

In [81]:
print len(gtop)

50


In [82]:
def get_top_genres_hash_values(data):
    gs = get_top_genres(data)
#     print len(gs)
    genres = set()
    for g in gs:
        genres.add(gethexdig(g[0])) 
    return genres

In [83]:
ghashed = get_top_genres_hash_values(data)
print len(ghashed)

50


In [84]:
itemsuserrated = defaultdict(lambda: list())
useritems = defaultdict(lambda: list())
for d in data:
    itemsuserrated[d[0]].append((d[1], d[3]))

for u in itemsuserrated:
    items = itemsuserrated[u]
    isorted = sorted(items, key = lambda tup: tup[1])
    itemsuserrated[u] = isorted
    useritems[u] = [i[0] for i in isorted]

In [85]:
print len(useritems)
print data[0]

10
('A13G1TKIKHGV3F', '0060518499', '4.0', '1237075200')


In [86]:
# ps = set()
# for d in data:
#     ps.add(d[1])
# with open('550 core subset - with items not rated.csv', 'wb') as f:
#     writer = csv.DictWriter(f, fieldnames=['user','irated','inrated'])
#     writer.writeheader()
#     for user in useritems:
#         itemsrated = useritems[user] 
#         for i in itemsrated:
#             notrated = np.random.choice(list(ps.difference(useritems[user])),size =10)
#             for n in notrated:
#                 writer.writerow({'user': user, 'irated': i, 'inrated': n})

In [87]:
# print itemsuserrated['A2OJW07GQRNJUT']

In [88]:
data = []
with open('500 core subset - with items not rated.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append((row['user'], row['irated'], row['inrated']))

In [89]:
def same_year(u1,u2):
    d1= datetime.fromtimestamp(float(u1))
    d2= datetime.fromtimestamp(float(u2))
    delta = d1 - d2
    return delta.days <=365 and delta.days >= 0

def reviews_before_in_same_year(uid, pid):
    pids = []
    reviews = useritems[uid]
    reviewswithtime = itemsuserrated[uid]
    index = list(reviews).index(pid)
    year = reviewswithtime[index][1]
    for i in range(index):
        if same_year(year, reviewswithtime[i][1]):
            pids.append(reviewswithtime[i][0])
    return pids

In [90]:
datag = []
depth = 3
dg = 'anything'
for d in data:
    cats = defaultdict(int)
    index = list(useritems[d[0]]).index(d[1])
    cgenre = getCategory(d[1],categories,catindex,depth)
    if gethexdig(cgenre) not in ghashed:
        cgenre = dg
    ngenre = getCategory(d[2], categories,catindex, depth)
    if gethexdig(ngenre) not in ghashed:
        ngenre = dg
    if not index==0:
        pids = reviews_before_in_same_year(d[0], d[1])
        for p in pids:
            cat = getCategory(p, categories, catindex, depth)
            if gethexdig(cat) not in ghashed :
                cat = dg
            cats[cat]+=1        
    tup = (d + (cgenre,ngenre))
    for c in cats:
        tup = tup + (c, cats[c])
    datag.append(tup)
    
k = 0
genres = {}
for g in gtop:
    genres[g[0]] = k
    k+=1
genres[dg] = k
len(genres)
gside = len(genres)

In [91]:
print datag[0]

('A2AYSFGUP5VTY3', '1856498050', '1401202209', 'anything', 'Graphic Novels', 'Humor', 2, 'Politics & Government', 2, 'Used & Rental Textbooks', 6, 'Fitness & Dieting', 1, 'Literary', 5, 'Music', 2, 'Education & Reference', 5, 'Religion & Spirituality', 1, 'History & Criticism', 2, 'Graphic Novels', 1, 'Ethnic & National', 2, 'Thriller & Suspense', 21, 'Biographies & Memoirs', 4, 'Economics', 1, 'Hobbies & Home', 2, 'Americas', 5, 'United States', 3, 'Leaders & Notable People', 3, 'Historical', 2, 'Classics', 10, 'anything', 29, 'People', 2, 'Arts & Literature', 4, 'Contemporary', 14, 'Books', 51, 'Literature & Fiction', 10)


In [92]:
print gside, len(genres)
print len(datag), len(data)
print datag[0]

51 51
8241 8241
('A2AYSFGUP5VTY3', '1856498050', '1401202209', 'anything', 'Graphic Novels', 'Humor', 2, 'Politics & Government', 2, 'Used & Rental Textbooks', 6, 'Fitness & Dieting', 1, 'Literary', 5, 'Music', 2, 'Education & Reference', 5, 'Religion & Spirituality', 1, 'History & Criticism', 2, 'Graphic Novels', 1, 'Ethnic & National', 2, 'Thriller & Suspense', 21, 'Biographies & Memoirs', 4, 'Economics', 1, 'Hobbies & Home', 2, 'Americas', 5, 'United States', 3, 'Leaders & Notable People', 3, 'Historical', 2, 'Classics', 10, 'anything', 29, 'People', 2, 'Arts & Literature', 4, 'Contemporary', 14, 'Books', 51, 'Literature & Fiction', 10)


In [93]:
ps = set()
us = set()
gs = set()
for d in datag:
    ps.add(d[1])
    ps.add(d[2])
    us.add(d[0])
    for i in range(5,len(d), 2):
        gs.add((d[i],d[3]))
        gs.add((d[i], d[4]))

In [94]:
print len(us)
print len(ps)
print len(gs)

10
7976
2601


In [95]:
users = {}
products = {}
i =0
j=0
for i,u in enumerate(us):
    users[u] = i
for j,p in enumerate(ps):
    products[p] = j

In [96]:
# # np.random.shuffle(datag)
# dtest = set()
# dvalid= set() #to choose best regualrization parameter and dimninsionality of factor vectors 
# dtestusers = defaultdict(int)
# dvalidusers = defaultdict(int)
# for r in datag:
#     cr = len(itemsuserrated[r[0]])
#     vc = int((cr * 30)/float(100))
#     if dtestusers[r[0]]< vc:
#         dtestusers[r[0]] +=1
#         dtest.add(r)
#     if r not in dtest and dvalidusers[r[0]]<vc:
#         dvalidusers[r[0]]+=1
#         dvalid.add(r)

# dtrain = set()
# for r in datag:
#     if r not in dtest and r not in dvalid:
#         dtrain.add(r)
# dtrain = list(dtrain)
# dtest = list(dtest)

In [97]:
x60 = int((len(datag) * 60) / float(100))
x20 = int((len(datag) * 20) / float(100))
dtrain = datag[:x60]
dvalid = datag[x60: x60+ x20]
end =  x60+ x20
dtest = datag[end:]
print len(dtrain), len(dvalid), len(dtest), len(datag)
print len(dtrain) + len(dvalid) + len(dtest) == len(datag)

4944 1648 1649 8241
True


In [98]:
print dtrain[0]

('A2AYSFGUP5VTY3', '1856498050', '1401202209', 'anything', 'Graphic Novels', 'Humor', 2, 'Politics & Government', 2, 'Used & Rental Textbooks', 6, 'Fitness & Dieting', 1, 'Literary', 5, 'Music', 2, 'Education & Reference', 5, 'Religion & Spirituality', 1, 'History & Criticism', 2, 'Graphic Novels', 1, 'Ethnic & National', 2, 'Thriller & Suspense', 21, 'Biographies & Memoirs', 4, 'Economics', 1, 'Hobbies & Home', 2, 'Americas', 5, 'United States', 3, 'Leaders & Notable People', 3, 'Historical', 2, 'Classics', 10, 'anything', 29, 'People', 2, 'Arts & Literature', 4, 'Contemporary', 14, 'Books', 51, 'Literature & Fiction', 10)


In [99]:
#balanced error rate
def BER(predictions, labels):
    tp =0
    fp=0
    tn=0
    fn=0
    for i,l in enumerate(labels):
        if l==1 and predictions[i]==l:
            tp+=1
        if l==0 and predictions[i]==l:
            tn+=1
        if l==1 and predictions[i]==0:
            fn+=1
        if l==0 and predictions[i]==1:
            fp+=1
    fpr = fp / float(fp+tn)
    fnr = fn/ float(tp+fn)
    return 0.5 * (fpr + fnr)

In [100]:
#function that takes a dataset and the optimized parameters and the diminsionality of the factors and returns the predictions for
#this dataset using the parmaters
def getPredictions_score(data, op,k):
    predictions = []
    betaio = op[:len(products)]
    end = len(products)
    bij = np.array(op[end: end+(gside**2)])
    betaij = bij.reshape(gside, gside)
    end = end + (gside**2)
    gs = np.array(op[end:])
    opp = gs.reshape(len(users)+len(products),k)
    gammus = opp[:len(users),:]
    gammis= opp[len(users):,:]
    for d in data:
        p1 = betaio[products[d[1]]] + gammus[users[d[0]]].dot(gammis[products[d[1]]])
        p2 = betaio[products[d[2]]] + gammus[users[d[0]]].dot(gammis[products[d[2]]])
        for i in range(5,len(d), 2):
            p1 += betaij[genres[d[i]]][genres[d[3]]] * np.log10(d[i+1])
            p2 += betaij[genres[d[i]]][genres[d[4]]]  * np.log10(d[i+1])
        predictions.append(sigmoid(p1))
        predictions.append(sigmoid(p2))
    return np.array(predictions)
    

In [101]:
#function that takes a dataset and the optimized parameters and the diminsionality of the factors and returns the predictions for
#this dataset using the parmaters
def getPredictions(data, op,k):
#     print len(data)
    predictions = []
    betaio = op[:len(products)]
    end = len(products)
    bij = np.array(op[end: end+(gside**2)])
    betaij = bij.reshape(gside, gside)
    end = end + (gside**2)
    gs = np.array(op[end:])
    opp = gs.reshape(len(users)+len(products),k)
    gammus = opp[:len(users),:]
    gammis= opp[len(users):,:]
    for d in data:
        p1 = betaio[products[d[1]]] + gammus[users[d[0]]].dot(gammis[products[d[1]]])
        p2 = betaio[products[d[2]]] + gammus[users[d[0]]].dot(gammis[products[d[2]]])
        for i in range(5,len(d), 2):
            p1 += betaij[genres[d[i]]][genres[d[3]]]  * np.log10(d[i+1])
            p2 += betaij[genres[d[i]]][genres[d[4]]]  * np.log10(d[i+1])
        predictions.append(sigmoid(p1)>0.5)
        predictions.append(sigmoid(p2)>0.5)
    return np.array(predictions)
    

In [102]:
print dtrain[0]

('A2AYSFGUP5VTY3', '1856498050', '1401202209', 'anything', 'Graphic Novels', 'Humor', 2, 'Politics & Government', 2, 'Used & Rental Textbooks', 6, 'Fitness & Dieting', 1, 'Literary', 5, 'Music', 2, 'Education & Reference', 5, 'Religion & Spirituality', 1, 'History & Criticism', 2, 'Graphic Novels', 1, 'Ethnic & National', 2, 'Thriller & Suspense', 21, 'Biographies & Memoirs', 4, 'Economics', 1, 'Hobbies & Home', 2, 'Americas', 5, 'United States', 3, 'Leaders & Notable People', 3, 'Historical', 2, 'Classics', 10, 'anything', 29, 'People', 2, 'Arts & Literature', 4, 'Contemporary', 14, 'Books', 51, 'Literature & Fiction', 10)


In [103]:
def AUC(data, op, k):
    count = 0
    predictions = getPredictions_score(data, op,k)
    for i in range(0, len(predictions), 2):
        if predictions[i] > predictions[i+1]:
            count +=1
#     print count, len(data)
    return count / float(len(data))     

In [104]:
#takes the data and returns the true labels for this dataset
def getLabels(data):
    labels = []
    for d in data:
        labels.append(1)
        labels.append(0)
    return labels

In [105]:
def sigmoid(z):
    try:
        f = sc.special.expit(z)
    except RuntimeWarning:
        f = 1/float(1 + bigfloat.exp(-z, bigfloat.precision(100)))
    return f

In [106]:
uintrain = set()
pintrain = set()
gintrain = set()
for d in dtrain:
    pintrain.add(d[1])
    pintrain.add(d[2])
    uintrain.add(d[0])
    for i in xrange(5,len(d), 2):
        gintrain.add((d[i], d[3]))
        gintrain.add((d[i], d[4]))

In [107]:
def compute_cost(initial,lam1, lam2, k, dataset):
    betai = np.array(initial[:len(products)])
    end = len(products)
    bij = np.array(initial[end: end+(gside**2)])
    betaij = bij.reshape(gside, gside)
    end = end + (gside**2)
    gammas = np.array(initial[end:])
    gss = gammas.reshape(len(users)+len(products), k)
    gammau = gss[:len(users),:]
    gammai = gss[len(users):, :]
    J = 0
    for d in dataset: 
        term = (betai[products[d[1]]] + gammau[users[d[0]]].dot(gammai[products[d[1]]])) - (betai[products[d[2]]] + gammau[users[d[0]]].dot(gammai[products[d[2]]]))
        for i in range(5,len(d), 2):
            term += (betaij[genres[d[i]]][genres[d[3]]] - betaij[genres[d[i]]][genres[d[4]]])  * np.log10(d[i+1])

        J+= np.logaddexp(0, -term)
    for u in us:
        J += lam1 * np.sum(np.square(gammau[users[u]]))
    for p in ps:
        J+= lam1 * ( np.square(betai[products[p]]) + np.sum(np.square(gammai[products[p]])))
    for g in gs:
        J+= lam2 * np.square(betaij[genres[g[0]]][genres[g[1]]])

    g = gradient(betai, betaij, gammau,gammai,lam1,lam2, k, dataset)
#     print J
    return J,g

In [108]:
def gradient(betai,betaij, gammau,gammai, lam1, lam2, k, dataset):
    gbi = np.zeros(len(products))
    ggu = np.zeros((len(users),k))
    ggi = np.zeros((len(products),k))
    ggij = np.zeros((gside,gside))
    
    for d in dataset:
        term = (betai[products[d[1]]] + gammau[users[d[0]]].dot(gammai[products[d[1]]])) - (betai[products[d[2]]] + gammau[users[d[0]]].dot(gammai[products[d[2]]]))
        
        for i in range(5,len(d), 2):
            term += (betaij[genres[d[i]]][genres[d[3]]] - betaij[genres[d[i]]][genres[d[4]]])  * np.log10(d[i+1])
        try:
            f = np.exp(-term) * sigmoid(term)
        except RuntimeWarning:
            f = float((bigfloat.exp(-term, bigfloat.precision(100))) * (sigmoid(term)))
        ggu[users[d[0]]] += (gammai[products[d[2]]] - gammai[products[d[1]]]) * f
        ggi[products[d[1]]]  += (-gammau[users[d[0]]]) * f
        ggi[products[d[2]]] += gammau[users[d[0]]] * f
        gbi[products[d[1]]] += (-1) *f
        gbi[products[d[2]]] += f
        for i in range(5,len(d), 2):
            ggij[genres[d[i]]][genres[d[3]]] += (-1) * f  * np.log10(d[i+1])
            ggij[genres[d[i]]][genres[d[4]]] += f  * np.log10(d[i+1])

    for u in us:
        ggu[users[u]] += 2 * lam1 * gammau[users[u]]
    for p in ps:
        gbi[products[p]] += 2 * lam1 * betai[products[p]]
        ggi[products[p]] += 2 * lam1 * gammai[products[p]]
    for g in gs:
        ggij[genres[g[0]]][genres[g[1]]] += 2 * lam2 * betaij[genres[g[0]]][genres[g[1]]]
    
    l = list(gbi) + list(ggij.flatten()) + list(ggu.flatten()) + list(ggi.flatten())

    return np.array(l)    

In [109]:
def cost(initial, k, dataset):
    betai = np.array(initial[:len(products)])
    end = len(products)
    bij = np.array(initial[end: end+(gside**2)])
    betaij = bij.reshape(gside, gside)
    end = end + (gside**2)
    gammas = np.array(initial[end:])
    gss = gammas.reshape(len(users)+len(products), k)
    gammau = gss[:len(users),:]
    gammai = gss[len(users):, :]
    J = 0
    for d in dataset: 
        term = (betai[products[d[1]]] +gammau[users[d[0]]].dot(gammai[products[d[1]]]))-(betai[products[d[2]]] +gammau[users[d[0]]].dot(gammai[products[d[2]]]))
        for i in range(5,len(d), 2):
            term += (betaij[genres[d[i]]][genres[d[3]]] - betaij[genres[d[i]]][genres[d[4]]])  * np.log10(d[i+1])

        J+= np.logaddexp(0, -term)
   
    return J

In [110]:
class ConvergedException(Exception):
    pass
def check(xk):
    global result
    global presult
    global maxauc
    global count
    global mincost
    global nit
    
    nit+=1

    ccost = cost(xk, k, dvalid)
    
    if ccost> mincost:
        if nit >=100:
            count +=1
            if count == 20:
                print 'here'
                result[:] = presult
                raise ConvergedException()
                return

    else:
        mincost = ccost
        presult[:] = xk
        count = 0
#     print ccost, mincost

#     fpr, tpr, _ = roc_curve(getLabels(dvalid),getPredictions_score(dvalid, xk, k))
#     cauc = auc(fpr,tpr)
#     if cauc < maxauc:
#         count +=1
#         if count == 10:
#             print 'here'
#             result = presult
#             raise ConvergedException()
#             return
#     else: 
#         maxauc = cauc
#         count = 0
#         presult[:] = xk
#     print cauc, maxauc, presult[0]
    return
    

In [111]:
k = 10
betai = np.random.normal(0,0.1, len(products))
betaij = np.random.normal(0,0.1, (gside,gside))
gammai = np.random.normal(0, 0.1, (len(products), k))
gammau = np.random.normal(0, 0.1, (len(users), k))
for u in users:
    if u not in uintrain:
        gammau[users[u]] = 0
for p in products:
    if p not in pintrain:
        gammai[products[p]]=0
        betai[products[p]]=0
for g in gs:
    if g not in gintrain:
        betaij[genres[g[0]]][genres[g[1]]] = 0
initial = list(betai) + list(betaij.flatten()) + list(gammau.flatten()) + list(gammai.flatten())

In [118]:
class ConvergedException(Exception):
    pass
def check_updated(xk):
    global result
    global presult
    global maxauc
    global count
    global mincost
    global nit
    
    nit+=1

#     ccost = cost(xk, k, dvalid)
    
#     if ccost> mincost:
#         count +=1
#         if count == 30:
#             print 'here'
#             result[:] = presult
#             raise ConvergedException()
#             return

#     else:
#         mincost = ccost
#         presult[:] = xk
#         count = 0
#     fpr, tpr, _ = roc_curve(getLabels(dvalid),getPredictions_score(dvalid, xk, k))
    cauc = AUC(dvalid, xk, k)
    if cauc < maxauc:
        count +=1
        if count == 30:
            print 'here'
            result[:] = presult
            raise ConvergedException()
    else: 
        maxauc = cauc
        count = 0
        presult[:] = xk
#     print maxauc
#     , AUC(dtest, xk, k)

In [119]:
# class ConvergedException(Exception):
#     pass
# def check_updated(xk):
#     global result
#     global presult
#     global maxauc
#     global count
#     global mincost
#     global nit
    
#     nit+=1

# #     ccost = cost(xk, k, dvalid)
    
# #     if ccost> mincost:
# #         count +=1
# #         if count == 30:
# #             print 'here'
# #             result[:] = presult
# #             raise ConvergedException()
# #             return

# #     else:
# #         mincost = ccost
# #         presult[:] = xk
# #         count = 0
# #     fpr, tpr, _ = roc_curve(getLabels(dvalid),getPredictions_score(dvalid, xk, k))
#     if count % 20 == 0:
#         cauc = AUC(dvalid, xk, k)
#         if cauc < maxauc:
# #             count +=1
# #             if count == 30:
#             print 'here'
#             result[:] = presult
#             raise ConvergedException()
#         else: 
#             maxauc = cauc
#             count = 0
#             presult[:] = xk
#         print cauc
#     count +=1

# #     print maxauc

In [120]:
lams1 = [0.01]
lams2 = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
for l1 in lams1:
    for l2 in lams2: 
        count = 0
        presult = []
        result = []
        params = []
        maxauc = 0
        mincost = 100000000000000
        nit = 0
        try:
            params, c,d = fmin_l_bfgs_b(compute_cost, x0=initial,args=(l1, l2, k, dtrain, ), disp = 0, callback=check_updated)
            params = presult
            print 'normal'
        except ConvergedException:
            params = result
        print 'converged after :' + str(nit)
        print str(l1) + ' -> '+ str(l2) + ' : '+ str(cost(params, k, dvalid)) + ' ' +  str(cost(params, k , dtest)) + ' ' + str(AUC(dvalid, params, k)) + ' ' + str(AUC(dtest, params, k))

here
converged after :95
0.01 -> 0 : 1288.79687203 1153.0717153 0.750606796117 0.770163735597
here
converged after :98
0.01 -> 1e-05 : 1310.26090245 1173.11109058 0.75182038835 0.770163735597
here
converged after :85
0.01 -> 0.0001 : 1337.72178958 1186.15308135 0.751213592233 0.768950879321
here
converged after :110
0.01 -> 0.001 : 1046.57081058 954.325940433 0.762742718447 0.770770163736
here
converged after :220
0.01 -> 0.01 : 788.873293958 750.997506183 0.783373786408 0.773802304427
here
converged after :66
0.01 -> 0.1 : 1127.82047751 1051.65430483 0.726334951456 0.73074590661
here
converged after :44
0.01 -> 1 : 832.10832928 799.440667064 0.718446601942 0.732565191025
here
converged after :43
0.01 -> 10 : 834.734553518 817.455501436 0.714805825243 0.733171619163
here
converged after :40
0.01 -> 100 : 966.658870811 962.098238393 0.68567961165 0.701637355973
here
converged after :37
0.01 -> 1000 : 1092.87593565 1091.96075978 0.619538834951 0.618556701031
here
converged after :34
0.01

In [147]:
lams1 = [0.001]
lams2 = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
for l1 in lams1:
    for l2 in lams2: 
        count = 0
        presult = []
        result = []
        params = []
        maxauc = 0
        mincost = 100000000000000
        nit = 0
        try:
            params, c,d = fmin_l_bfgs_b(compute_cost, x0=initial,args=(l1, l2, k, dtrain, ), disp = 0, callback=check_updated)
            params = presult
            print 'normal'
        except ConvergedException:
            params = result
        print 'converged after :' + str(nit)
        print str(l1) + ' -> '+ str(l2) + ' : '+ str(cost(params, k, dvalid)) + ' ' +  str(cost(params, k , dtest)) + ' ' + str(AUC(dvalid, params, k)) + ' ' + str(AUC(dtest, params, k))

here
converged after :180
0.001 -> 0 : 57.5285679506 59.651989062 0.999450750641 0.999221896741
here
converged after :184
0.001 -> 1e-05 : 39.4175757403 40.6297451519 0.99967960454 0.999542292201
here
converged after :252
0.001 -> 0.0001 : 18.0247646876 19.0062402377 0.99986268766 0.99986268766
here
converged after :212
0.001 -> 0.001 : 14.0539212384 16.6328567549 0.99995422922 0.99986268766
here
converged after :314
0.001 -> 0.01 : 12.5329634224 12.2363288325 1.0 0.99995422922
here
converged after :125
0.001 -> 0.1 : 24.4118649628 31.3441962585 0.99990845844 0.99972537532
here
converged after :225
0.001 -> 1 : 25.8060611734 31.4512118142 0.99986268766 0.99967960454
here
converged after :322
0.001 -> 10 : 33.5252134667 45.6853829185 0.99986268766 0.999633833761
here
converged after :1019
0.001 -> 100 : 25.2687995218 29.6938471607 0.99990845844 0.9997711461
here
converged after :1040
0.001 -> 1000 : 267.851757768 218.680476774 0.998581105822 0.998672647382
here
converged after :216
0.00

In [147]:
# lams = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10]
lams1 = [0.001]
lams2 = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
with open('Binary baseline - Pairwise genre bias terms (Adjacent) 400 core.csv', 'wb') as f:
    writer = csv.DictWriter(f, fieldnames=['lambda1', 'lambda2','k','value'])
    writer.writeheader()
    for l1 in lams1:
         for l2 in lams2:
            count = 0
            nit = 0
            presult = []
            result = []
            maxauc = 0
            params = []
            mincost = 100000000
            try:
                params, c,d = fmin_l_bfgs_b(compute_cost, x0=initial,args=(l1, l2, k, dtrain, ), disp = 0, callback = check)
                params = presult
            except ConvergedException:
                params = result
            print 'converged after :' + str(nit)
            fpr, tpr, _ = roc_curve(getLabels(dvalid),getPredictions_score(dvalid, params, k))
            print str(l1) + ' -> ' + str(l2) + ' : ' +  str(auc(fpr, tpr))

#             print str(l1) + ' -> ' + str(l2) + ' : ' +  str(cost(params, k, dvalid))
            for i, v in enumerate(params):
                writer.writerow({'lambda1': l1, 'lambda2': l2, 'k': k, 'value': v}) 

here
converged after :145
0.001 -> 0 : 0.999179735585
here
converged after :203
0.001 -> 1e-05 : 0.99960204163
here
converged after :215
0.001 -> 0.0001 : 0.999903883038
here
converged after :191
0.001 -> 0.001 : 0.999884449102
here
converged after :180
0.001 -> 0.01 : 0.999875461705
here
converged after :186
0.001 -> 0.1 : 0.999877538862
here
converged after :236
0.001 -> 1 : 0.999863228161
here
converged after :623
0.001 -> 10 : 0.999884632411
here
converged after :119
0.001 -> 100 : 0.993877891076
here
converged after :264
0.001 -> 1000 : 0.991662602506
here
converged after :279
0.001 -> 10000 : 0.976040141602


In [148]:
print len(initial)

116732


In [149]:
params = defaultdict(lambda:defaultdict(list))
with open('Binary baseline - Pairwise genre bias terms (Adjacent) 400 core.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        params[(float(row['lambda1']), float(row['lambda2']))][int(row['k'])].append(float(row['value']))

In [150]:
print len(params)

11


In [151]:
#takes dictionary of parameters in the form of  {lambda value: k value: [parameters]} and choose the lambda value and k value that
#minimize the balanced error rate
def chooseBestParams(data, params):
    minError = 1000000000
    maxauc = 0
    labs = getLabels(data)
    for l in params:
        for k in params[l]:
#             fpr, tpr, _ = roc_curve(getLabels(dvalid),getPredictions_score(dvalid, params[l][k], k))
#             cauc = auc(fpr,tpr)
#             if cauc > maxauc:
#                 maxauc = cauc
#                 minlam = l
#                 op = params[l][k]


            e = cost(params[l][k], k, data)
#             print l, e
#             print l, k , e
            if e < minError:
                minError = e
                minlam = l
                op = params[l][k]
    return minlam, op, minError

In [152]:
minlam, op, e = chooseBestParams(dvalid, params)
print minlam,e 

(0.001, 0.001) 9.93178579845


In [153]:
print cost(op, k, dtest)

10.5266741589


In [154]:
print cost(op, k, dtrain)

0.743070239412


In [155]:
fpr, tpr, _ = roc_curve(getLabels(dtest),getPredictions_score(dtest, op, k))
roc_auc = auc(fpr,tpr)
print roc_auc

0.999900908189


In [156]:
fpr, tpr, _ = roc_curve(getLabels(dtrain),getPredictions_score(dtrain, op, k))
roc_auc = auc(fpr,tpr)
print roc_auc

0.999999985335


In [157]:
preds = getPredictions(dtest, op, k)
labels = getLabels(dtest)
print BER(preds, labels)

0.156993775174


In [ ]:
# from sklearn.metrics import roc_curve, auc
# import matplotlib.pyplot as plt
# plt.figure()
# plt.plot(fpr, tpr, label='ROC curve (area = %0.5f)' % roc_auc)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve Adjacent')
# plt.legend(loc="lower right")
# plt.show()

In [192]:
# labs = getLabels(dtest)
# preds = getPredictions(dtest, op,mink)
# e = BER(preds,labs)
# print e

In [31]:
# mink = 1
# minlam = 10
# betai = np.random.normal(0,0.1, len(products))
# betaij = np.random.normal(0,0.1, (gside, gside))
# gammai = np.random.normal(0, 0.01, (len(products), mink))
# gammau = np.random.normal(0, 0.01, (len(users), mink))
# initial = list(betai.flatten())+ list(betaij.flatten()) + list(gammau.flatten()) + list(gammai.flatten())
# grad = gradient(betai,betaij, gammau, gammai, minlam,mink, dtrain[:1])
# numgrad = computeNumericalGradient(initial,grad, minlam, mink, dtrain[:1])

In [54]:
def compute_cost1(initial,lam,k, dataset):
    betai = np.array(initial[:len(products)])
    end = len(products)
    bij = np.array(initial[end: end+(gside**2)])
    betaij = bij.reshape(gside, gside)
    end = end + (gside**2)
    gammas = np.array(initial[end:])
    gss = gammas.reshape(len(users)+len(products), k)
    gammau = gss[:len(users),:]
    gammai = gss[len(users):, :]
    J = 0
    for d in dataset: 
        term = (betai[products[d[1]]] +gammau[users[d[0]]].dot(gammai[products[d[1]]]))-(betai[products[d[2]]] +gammau[users[d[0]]].dot(gammai[products[d[2]]]))
        for i in range(5,len(d)):
            term += betaij[genres[d[i]]][genres[d[3]]] - betaij[genres[d[i]]][genres[d[4]]]

        J+= np.logaddexp(0, -term)
    for u in us:
        J += lam * np.sum(np.square(gammau[users[u]]))
    for p in ps:
        J+= lam * ( np.square(betai[products[p]]) + np.sum(np.square(gammai[products[p]])))
    for g in gs:
        J+= lam * np.square(betaij[genres[g[0]]][genres[g[1]]])

#     print J
    return J

In [55]:
def gradient1(initial, lam,k, dataset):
    betai = np.array(initial[:len(products)])
    end = len(products)
    bij = np.array(initial[end: end+(gside**2)])
    betaij = bij.reshape(gside, gside)
    end = end + (gside**2)
    gammas = np.array(initial[end:])
    gss = gammas.reshape(len(users)+len(products), k)
    gammau = gss[:len(users),:]
    gammai = gss[len(users):, :]
    gbi = np.zeros(len(products))
    ggu = np.zeros((len(users),k))
    ggi = np.zeros((len(products),k))
    ggij = np.zeros((gside,gside))
    
    for d in dataset:
        term = (betai[products[d[1]]] + gammau[users[d[0]]].dot(gammai[products[d[1]]])) -(betai[products[d[2]]] + gammau[users[d[0]]].dot(gammai[products[d[2]]]))    
        
        for i in range(5,len(d)):
            term += betaij[genres[d[i]]][genres[d[3]]] - betaij[genres[d[i]]][genres[d[4]]]
        try:
            f = np.exp(-term) * sigmoid(term)
        except RuntimeWarning:
            f = float((bigfloat.exp(-term, bigfloat.precision(100))) * (sigmoid(term)))
        ggu[users[d[0]]] += (gammai[products[d[2]]] - gammai[products[d[1]]]) * f
        ggi[products[d[1]]]  += (-gammau[users[d[0]]]) * f
        ggi[products[d[2]]] += gammau[users[d[0]]] * f
        gbi[products[d[1]]] += (-1) *f
        gbi[products[d[2]]] += f
        for i in range(5,len(d)):
            ggij[genres[d[i]]][genres[d[3]]] += (-1) * f
            ggij[genres[d[i]]][genres[d[4]]] += f

    for u in us:
        ggu[users[u]] += 2 * lam * gammau[users[u]]
    for p in ps:
        gbi[products[p]] += 2 * lam * betai[products[p]]
        ggi[products[p]] += 2 * lam * gammai[products[p]]
    for g in gs:
        ggij[genres[g[0]]][genres[g[1]]] += 2 * lam * betaij[genres[g[0]]][genres[g[1]]]
    
    l = list(gbi) + list(ggij.flatten()) + list(ggu.flatten()) + list(ggi.flatten())

    return np.array(l)    

In [62]:
def computeNumericalGradient(params,grad, l1, l2, k, data):
    e = 1e-4
    numgrads = np.zeros(len(params))
    pp = np.zeros(len(params))
    for i,p in enumerate(params):
        pp[i] = e
        loss1, _ = compute_cost(params-pp, l1, l2, k,data)
        loss2, _ = compute_cost(params+pp, l1,l2,k, data)
#         print loss1, loss2
        numgrads[i] = (loss2 - loss1) / float(2*e)
        pp[i] = 0
        print round(numgrads[i], 3), round(grad[i], 3)
#         print abs(numgrads[i]-grad[i])
    return numgrads

In [ ]:
# _,g = compute_cost(initial, 0.1, 1, 10, dtrain[:1])
# numgrad = computeNumericalGradient(initial,g ,0.1, 1 , 10 , dtrain[:1] )

In [57]:
diff = np.linalg.norm(numgrad-g)/np.linalg.norm(numgrad+g);

In [58]:
print diff < 1e-9

True


In [59]:
print diff

1.2923313768e-10


In [ ]:
print compute_cost(params, 0, 10, dtrain)